#PyTorch Intro - Tensory - Wykład

**Tensor** w bibliotece PyTorch jest specjalizowaną strukturą danych podobną do wielowymiarowej tablicy `ndarray` w bibliotece numpy. Na tensorach można wykonywać typowe operacje algebry liniowej oraz operacje specyficzne dla głębokiego uczenia jak na przykład automatyczne różniczkowanie (autograd).
Tensory wykorzystujemy do przechowywania przetwarzanych danych oraz parametrów (wag) modeli. Biblioteka PyTorch pozwala na efektywne przetwarzanie tensorów na CPU lub GPU.

##Przygotowanie środowiska
Upewnij się, że notatnik jest uruchomiony na maszynie z GPU. Jeśli GPU nie jest dostępne zmień typ maszyny (Runtime | Change runtime type) i wybierz T4 GPU.

In [ ]:
!nvidia-smi

Biblioteka PyTorch (`torch`) jest domyślnie zainstalowana w środowisku COLAB.

In [ ]:
import torch
import numpy as np

print(f"Wersja biblioteki PyTorch: {torch.__version__}")

Sprawdzenie dostępnego urządzenia GPU.

In [ ]:
print(f"Dostępność GPU: {torch.cuda.is_available()}")
print(f"Typ GPU: {torch.cuda.get_device_name(0)}")

##Tworzenie tensorów

Funkcja `tensor()` ([link](https://pytorch.org/docs/stable/generated/torch.tensor.html#torch-tensor)) **konstruuje tensor z podanych danych** na domyślnym urządzeniu (CPU jeśli nie podano inaczej). Typ utworzonego tensora, o ile nie podamy inaczej, odpowiada typowi danych źródłowych. W tym przypadku utworzony został tensor przechowujący dane typu `torch.float32` (32-bitowa liczba zmiennoprzecinkowa).

In [ ]:
data = [[1.1, 2.1, 3.5],[4.2, 5.7, 0.6],[3.2, 0.7, 1.7]]

x = torch.tensor(data)
print(f"{x=}")
print(f"{type(x)=}")
print(f"{x.dtype=}")


**Wybrane atrybuty i metody tensorów**:
*   `dtype` - typ tensora (lista dostępnych typów: [link](https://pytorch.org/docs/stable/tensor_attributes.html#torch.dtype))
*   `device` - urządzenie na którym jest przechowywany i będzie przetwarzany tensor ([link](https://pytorch.org/docs/stable/tensor_attributes.html#torch.device))
*   `layout` - format pamięci w jakim przechowywana jest zawartość tensora ([link](https://pytorch.org/docs/stable/tensor_attributes.html#torch.layout))
*   `shape` - rozmiar każdego wymiaru tensora (to samo co metoda `size()`)
*   `ndim` - liczba wymiarów tensora
*   `numel()` - liczba elementów tensora
*   `size()` - rozmiar każdego wymiaru tensora

In [ ]:
print(f"{x.dtype=}")
print(f"{x.device=}")
print(f"{x.layout=}")
print(f"{x.ndim=}")
print(f"{x.numel()=}")
print(f"{x.shape=}")
print(f"{x.size()=}")

Funkcja `torch.from_numpy(ndarray)` **tworzy tensor z tablicy `ndarray` z biblioteki numpy**. Uwaga: Tak utworzony tensor korzysta z tego samego obszaru pamięci co źródłowa tablica `ndarray`.

In [ ]:
array = np.array([[1, 7, 3], [2, 5, 11]])
print(array)
x = torch.from_numpy(array)
print(f"{x=}")
print(f"{type(x)=}")
print(f"{x.dtype=}")
print(f"{x.device=}")
print(f"{x.shape=}")

Przydatne funkcje tworzące zainicjalizowane tensory:
*   `torch.zeros(size, ...)` - tensor wypełniony zerami
*   `torch.ones(size, ...)` - tensor wypełniony jedynkami
*   `torch.full(size, fill_value, ...)` - tensor wypełniony podaną wartością
*   `torch.rand(size, ...)` - tensor o wartościach losowych o rozkładzie jednostajnym z przedziału $[0,1)$
*   `torch.randn(size, ...)` - tensor o wartościach losowych o standardowym rozkładzie normalnym
*   `torch.randint(low, high, size, ...)` - tensor losowych liczb całkowitych z przedziału `[low, high)`
*   `torch.arange(start=0, end, step=1, ...)` - jednowymiarowy tensor (wektor) o wartościach z przedziału `[start, end)` z krokiem  `step`

Opcjonalne parametry `dtype` i `device` pozwalają określić typ danych i urządzenie na którym przechowywany będzie tensor.

In [ ]:
size = (2, 3)

zeros_tensor = torch.zeros(size)
ones_tensor = torch.ones(size, device = "cuda", dtype=torch.int64)
full_tensor = torch.full(size, 3.14)
rand_tensor = torch.rand(size, device = "cuda")
randn_tensor = torch.rand(size)
randint_tensor = torch.randint(0, 100, size)
x = torch.arange(2, 10)

print(f"Tensor zer: \n {zeros_tensor} \n")
print(f"Tensor jedynek: \n {ones_tensor} \n")
print(f"Tensor losowy (rozkład równomierny na [0,1)): \n {rand_tensor} \n")
print(f"Tensor losowy (standardowy rozkład normalny): \n {randn_tensor} \n")
print(f"Tensor losowy (liczby całkowite): \n {randint_tensor} \n")
print(f"torch.arange: \n {x} \n")

Funkcje z rozszerzeniem `_like` (`torch.zeros_like`, `torch.ones_like`, `torch.full_like`, `torch.rand_like`, ...) domyślnie tworzą tensory o identycznych własnościach (rozmiary, typ danych, urządzenie, ...) jak tensory podane jako argument.

In [ ]:
x_source = torch.rand(2, 4, device="cuda")
print(x_source)
print()
x_ones = torch.ones_like(x_source)
print(x_ones)
print()
x_zeros = torch.zeros_like(x_source)
print(x_zeros)

##Operacje na tensorach

### Zmiana typu lub urządzenia

Metoda `to` pozwala zmienić typ tensora lub urządzenie na którym będzie przetwarzany tensor. Metoda `cpu` przesyła tensor na CPU.


In [ ]:
x = torch.randint(0, 100, (3, 2))
print(x)
print(f"{x.dtype=}   {x.device=}\n")

cuda_device = torch.device('cuda:0')
y = x.to(cuda_device)
print(y)
print(f"{y.dtype=}   {y.device=}\n")

y = y.to(torch.float32)
print(y)
print(f"{y.dtype=}   {y.device=}\n")

z = y.to('cpu')
print(f"{z.dtype=}   {z.device=}\n")

z = y.cpu()
print(f"{z.dtype=}   {z.device=}\n")


###Indeksowanie tensorów

Biblioteka PyTorch posiada podobny **mechanizm indeksowania tensorów** jak w bibliotece numpy.
Indeksy elementów tensora wzdłuż każdej osi rozpoczynają się od `0`.

Indeksy ujemne pozwalają odwołać się do elementu liczonego od końca (`x[-1]` oznacza ostatni element wektora `x`, `x[-2]` element przedostatni).

Podobnie jak w bibliotece numpy wspierane jest indeksowanie przez podanie zakresu (*slicing*).
`x[start:stop]` oznacza zakres od indeksu `start` do indeksu `stop` ale BEZ elementu o indeksie `stop`. Np. x[2:4] uwzględnia tylko elementy o indeksach 2 i 3.
`x[start:]` oznacza elementy wektora `x` od indeksu `start` do końca.
`x[:stop]` oznacza początkowe elementy wektora x do indeksu `stop` (ale BEZ elementu o indeksie `stop`).
Sam dwukropek oznacza wszystkie elementy w danym wymiarze, np. `x[:, 2:5]` odwołuje się do wszystkich wierszy i kolumn 2, 3 i 4 tensora `x`.

Przeczytaj opis indeksowania w bibliotece numpy: [link](https://numpy.org/doc/stable/user/basics.indexing.html).

In [ ]:
x = torch.randint(0, 100, (4, 4))
print(f"{x=}")
print()

x[:,1] = 0
print(f"{x=}")
print()

print(f"{x[..., 3]=}")
print()

print(f"{x[1:3, 2:4]=}")
print()

print(f"{x[-1]=}")
print()
print(f"{x[-2]=}")
print()

print(f"{x[:, -1]=}")
print()

### Rozgłaszanie (*broadcasting*)

Biblioteka PyTorch wspiera **rozgłaszanie (brodcasting)**, czyli dostosowywanie wymiarów tensorów, podobnie jak w bibliotece numpy.
Rozgłaszanie pozwala pod pewnymi warunkami wykonać operacje na tensorach o niezgodnych wymiarach.

**Zasady uzgadniania rozmiarów tensorów**:
*   Jeśli jeden z tensorów ma mniejszą liczbę wymiarów niż drugi, jest niejawnie rozszerzany poprzez dodanie jednostkowych wymiarów z przodu
*   Jeśli dwa tensory mają różny rozmiar któregoś z wymiarów:
    *   Jeśli jeden z nich ma rozmiar jeden, to jest niejawnie rozszerzany poprzez powielenie wartości wzdłuż tej osi
    *   Jeśli żaden z nich nie ma rozmiaru jeden – uzgodnienie wymiarów kończy się niepowodzeniem

Przeczytaj opis rozgłaszania w bibliotece numpy: [link](https://numpy.org/doc/stable/user/basics.broadcasting.html).

W poniższym przykładzie tensory `x` i `y` mają niezgodne rozmiary (odpowiednio $3 \times 1$ i $1 \times 2$). Przed wykonaniem dodawania mechanizm rozgłaszania sprowadza je do zgodnych rozmiarów ($3 \times 2$) poprzez powielenie jednostkowych wymiarów.

In [ ]:
x = torch.arange(3).reshape((3, 1))
y = torch.arange(10,12).reshape((1, 2))
print(f"{x=}")
print(f"{y=}")
print()

print(f"{x + y=}")

In [ ]:
x = torch.empty(4, 2).normal_(2)
print(f"{x=}")
y = x.mean(dim=0)
print(f"{y=}")

print(f"{x - y=}")

Zwróć uwagę na wynik dodawania w tym przykładzie.
W pierwszym kroku tensor `y` o mniejszej liczbie wymiarów zostanie niejawnie rozszerzony do rozmiaru $(1,4)$ poprzez dodanie jednostkowego wymiaru z przodu.

In [ ]:
x = torch.arange(4).reshape((4, 1))
y = torch.arange(4).reshape((4))
print(f"{x=}")
print(f"{y=}")
print()

print(f"{x + y=}")

###Przekształcanie tensorów

Przydatne operacje przekształcające i zmieniające rozmiary tensora:

*   `squeeze` - usunięcie wybranego wymiaru rozmiaru 1
*   `unsqueeze` - dodanie wymiaru rozmiaru 1 na wybranej pozycji
*   `permute` - permutacja kolejności wymiarów
*   `reshape` - zmiana kształtu (rozmiaru wymiarów) tensora; wynikowy tensor musi zawierać tyle samo elementów co wejściowy tensor
*   `view`- podobnie jak `reshape`, ale zwraca widok wejściowego tensora
*   `contiguous` - zwraca tensor identyczny z wejściowym, ale zajmujący ciągły obszar pamięci


Powyższe operacje można wywoływać jako funkcje biblioteki torch (np. `torch.squeeze(x, dim=1)`, albo jako metody klasy `torch.Tensor` (np. `x.squeeze(dim=1)`).

In [ ]:
x = torch.randn((10, 1, 5,2))
print(f"{x.shape=}")

x = x.squeeze(dim=1)
print(f"Po wykonaniu x.squeeze(dim=1) {x.shape=}")

x = x.unsqueeze(dim=0)
print(f"Po wykonaniu x.unsqueeze(dim=0) {x.shape=} \n")

print(f"{x.is_contiguous()=}")
x = x.permute(0, 3, 1, 2)
print(f"Po wykonaniu x.permute(0, 3, 1, 2) {x.shape=}")
print(f"{x.is_contiguous()=}")
x = x.contiguous()
print(f"Po wykonaniu x.contiguous() {x.is_contiguous()=}")

x = x.reshape((50, 2))
print(f"Po wykonaniu x.reshape((50, 2)) {x.shape=}")

# Nieprawidłowa operacja
# Wynikowy tensor musi zawierać tyle samo elementów co wejściowy tensor
x = x.reshape((50, 4))

Funkcja `torch.cat` umożliwia konkatenację tensorów wzdłuż podanego wymiaru.
Inną funkcją umożliwiającą scalanie wielu tensorów jest `torch.stack`. W odróżnieniu od `torch.cat` łączy listę tensorów tworząc nowy wymiar.

In [ ]:
print(f"{x.shape=}")
print(f"{x.ndim=}")
print()

t1 = torch.cat([x, x, x], dim=1)
print(f"{t1.shape=}")
print(f"{t1.ndim=}")
print()

t2 = torch.stack([x, x, x], dim=1)
print(f"{t2.shape=}")
print(f"{t2.ndim=}")


###Operacje matematyczne

Metoda `t()` zwraca macierz transponowaną.

In [ ]:
m = torch.rand((2, 3))
print(m)
print()
print(m.t())

Funkcja `mul` i operator `*` oblicza iloczyn po współrzędnych (iloczyn Hadamarda, oznaczany $\odot$) macierzy.
Funkcja `matmul` i operator `@` oblicza standardowy iloczyn macierzy.


In [ ]:
m = torch.rand((2, 3))
print(m)

print("\nIloczyn po współrzędnych m * m")
print(m * m)

print("\nStandardowy iloczyn macierzy m @ m.t()")
print(m @ m.t())

Metody z postfiksem `_` w nazwie wykonują się w miejscu (*in-place*).

In [ ]:
print(m, "\n")
m.add_(5)
print(m)

**Uwaga:** Tensory będące argumentami funkcji muszą znajdować się na tym samym urządzeniu.

In [ ]:
x = torch.ones((4, 4) , device='cpu')
y = torch.ones((4, 4), device='cpu')

print(x + y)

y = y.to('cuda')
print(x + y)
# Błąd, tensory na różnych urządzeniach

W bibliotece PyTorch zaimplementowano wiele operacji matematycznych na tensorach:
- Operacje arytmetyczne: `abs`, `ceil`, `floor`, `exp`, `log`, `trunc`, `sign`, `sqrt`, ...
- Funkcje trygonometryczne: `sin`, `cos`, `tan`, `asin`, `acos`, `atan`, ...
- Operacje na bitach: `bitwise_not`, `bitwise_and`, `bitwise_or`, `bitwise_xor`, `bitwise_left_shift`, ...
- Operacje logiczne: `logical_not`, `logical_and`, `logical_or`, ...

Listę dostępnych operacji matematycznych znajdziesz tutaj: [link](https://pytorch.org/docs/stable/torch.html#math-operations).

Inne operacje na tensorach:
*   Operacje redukcji: `argmax`, `argmin`, `max`, `min`, `mean`, `std`, `median`, `mode`, `quantile`, `norm`, `sum`, `any`, `all`, `count`, `unique`. Pełna lista: [link](https://pytorch.org/docs/stable/torch.html#reduction-ops).
*   Operacje sortowania i porównywania: `argsort`, `kthvalue`, `sort`, `topk`. Pełna lista: [link](https://pytorch.org/docs/stable/torch.html#comparison-ops).
*   Metody algebry liniowej: `inv`, `det`, `eig`, `svd`, `lstsq`. Pełna lista: ([link](https://pytorch.org/docs/stable/linalg.html)).


In [ ]:
x = torch.rand((3,5)) * 10 - 5
print(x)

# Operacje redukcji po wszystkich elementach tensora
print(f"{x.min()=}")
print(f"{x.max()=}")
print(f"{x.mean()=}")
print(f"{x.sum()=}")
print(f"{x.sum()=}")
print(f"{x.norm()=}")
print()
# Operacje reukcji po okreslonym wymiarze
print(f"{x.min(dim=1)=}")
print(f"{x.max(dim=0)=}")
print(f"{x.mean(dim=0)=}")
print(f"{x.sum(dim=1)=}")
print(f"{x.norm(dim=1)=}")

###Konwencja sumacyjna Einsteina

Konwencja sumacyjna Einsteina to sposób zapisu operacji na macierzach i tensorach.
Pozwala zwięźle zapisać złożone operacje na wielowymiarowych tablicach.
Podstawowe zasady konwencji Einsteina:

*   Indeksy (np. $i$, $j$, $k$) reprezentują osie tensora (np. wiersze, kolumny, głębokość w przypadku 3D) używane w obliczeniach.

*   Wynik jest obliczany przez **zsumowanie iloczynu elementów** o podanych indeksach **po osiach których indeksy nie są podane w wyrażeniu wynikowym**.

W PyTorch funkcja `torch.einsum` wykonuje wyrażenia zapisane w konwencji Einsteina. Na przykład mnożenie macierzy `A` i `B` może zostać zapisane jako `torch.einsum('ik,kj->ij', A, B)`. W tym przypadku sumowanie odbywa się po indeksie `k` (ponieważ nie występuje w wyrażeniu wynikowym) a wynikowa macierz jest indeksowana `i` i `j` (czyli ma tyle wierszy ile macierz `A` i tyle kolumn ile macierz `B`).

####Przykłady

**Iloczyn skalarny** wektorów $\mathbf{a}=[a_1,\ldots, a_n]$ i $\mathbf{b}=[b_1,\ldots, b_n]$ jest zdefiniowany jako: $$c=\sum_i a_i b_i $$.



In [ ]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])

# Iloczyn skalarny
# Sumowanie po indeksie i ponieważ nie występuje w wynikowym wyrażeniu
# Wynikiem jest skalar, poniważ w wynikowym wyrażeniu nie ma indeksów
result = torch.einsum('i,i->', a, b)
print(result)

**Mnożenie macierzy** $\mathbf{A}$ i $\mathbf{B}$ jest zdefiniowane jako: $$C_{ij}=\sum_k A_{ik} B_{kj}$$

In [ ]:
A = torch.tensor([[1, 2], [3, 4]])
B = torch.tensor([[5, 6], [7, 8]])

C = torch.einsum('ik,kj->ij', A, B)
# Sumowanie po indeksie k ponieważ nie występuje w wynikowym wyrażeniu

print(C)

**Iloczyn zewnętrzny** wektorów $\mathbf{a}=[a_1,\ldots, a_n]$ i $\mathbf{b}=[b_1,\ldots, b_n]$ jest zdefiniowany jako: $$C_{ij}=a_i b_j $$.


In [ ]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5])

C = torch.einsum('i,j->ij', a, b)
print(C)

###Porównanie czasów działania CPU versus GPU

Porównanie czasów mnożenia dużych macierzy na CPU i GPU.

In [ ]:
import time

In [ ]:
def measure_time(device, size=1000):
    # Utwórz losowe macierze
    a = torch.rand(size, size, device=device)
    b = torch.rand(size, size, device=device)

    if device.type == "cuda":
        #Wait for all kernels in all streams on a CUDA device to complete.
        torch.cuda.synchronize()
    start_time = time.time()

    # Mnożenie macierzy
    c = torch.matmul(a, b)

    if device.type == "cuda":
        #Wait for all kernels in all streams on a CUDA device to complete.
        torch.cuda.synchronize()
    end_time = time.time()

    return end_time - start_time

In [ ]:
size = 10000
device_cpu = torch.device("cpu")
device_gpu = torch.device("cuda")

print(f"Rozmiar macierzy: {size}x{size}")
cpu_time = measure_time(device_cpu, size)
print(f"Czas wykonania na CPU: {cpu_time:.6f} s.")

gpu_time = measure_time(device_gpu, size)
print(f"Czas wykonania na GPU: {gpu_time:.6f} s.")

# Calculate speedup
speedup = cpu_time / gpu_time
print(f"Przyśpieszenie (CPU/GPU): {speedup:.2f}x")

##Przykłady

###Analiza głównych składowych (PCA)

Implementacja metody analizy głównych składowych (*principal component analysis*) w PyTorch.
Opis metody PCA: [link](https://pl.wikipedia.org/wiki/Analiza_g%C5%82%C3%B3wnych_sk%C5%82adowych).

In [ ]:
import matplotlib.pyplot as plt

# Wygeneruj silnie skorelowane syntetyczne dane
num_samples = 1000
mean = [5, 10]
cov = [[3, 2], [2, 2]]      # Macierz kowariancji
data = np.random.multivariate_normal(mean, cov, num_samples)

# Przekształć na tensor
data_tensor = torch.tensor(data, dtype=torch.float32, device='cuda')

In [ ]:
# Krok 1: Centrowanie danych
data_mean = data_tensor.mean(dim=0)
centered_data = data_tensor - data_mean

# Krok 2: Wyznaczenie macierzy kowariancji
cov_matrix = torch.mm(centered_data.T, centered_data) / (num_samples - 1)

# Krok 3: Dekompozycja względem wartości własnych
# Macierz kowariancji jest symetryczna - możemy użyć eigh zamiast eig
eigenvalues, eigenvectors = torch.linalg.eigh(cov_matrix)
print(f"Wartości własne macierz kowariancji: {eigenvalues}")
print(f"Wektory własne macierz kowariancji: \n{eigenvectors}")

# Step 4: Posortuj względem malejących wartości własnych
sorted_indices = torch.argsort(eigenvalues, descending=True)
sorted_eigenvalues = eigenvalues[sorted_indices]
sorted_eigenvectors = eigenvectors[:, sorted_indices]

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(data[:, 0], data[:, 1], alpha=0.5, label="Dane wejściowe")
origin = data_mean.cpu().numpy()  # Punkt zaczepienia wektorów
for i in range(sorted_eigenvectors.shape[1]):
    vector = sorted_eigenvectors[:, i].cpu().numpy()
    scale = sorted_eigenvalues[i].sqrt().item()
    plt.quiver(origin[0], origin[1], vector[0] * scale, vector[1] * scale,
               angles='xy', scale_units='xy', scale=1)

plt.xlabel("Cecha 1")
plt.ylabel("Cecha 2")
plt.title("Dystrybucja danych")
plt.show()

###Regresja liniowa

Niech dany będzie zbiór obserwacji (punktów) $(x_n, y_n) \in \mathbb{R}^2, n=1, \ldots, N$.
Celem jest wyznaczenie współczynników funkcji liniowej najlepiej dopasowanej do zbioru obserwacji.
Przykładowy zbiór obserwacji `data`  zawiera pary (wiek pacjenta, zmierzone skurczowe ciśnienie krwi).

In [ ]:
data = [(39, 144), (47, 220), (45, 138), (47, 145), (65, 162), (46, 142),
        (67, 170), (42, 124), (67, 158), (56, 154), (64, 162), (56, 150),
        (59, 140), (34, 110), (42, 128)]

Problem możemy sformalizować jako znalezienie współczynników funkcji $f(x) = ax+b$ minimalizujących błąd średniokwadratowy:
$$
\frac{1}{N}\sum_{n=1}^{N} \left( f \left(x_n\right) - y_n \right)^2
=
\frac{1}{N}\sum_{n=1}^{N} \left( a x_n + b - y_n \right)^2
\, .$$
W postaci macierzowej problem możemy przedstawić, jako znalezienie przybliżonego rozwiązania nadokreślonego układu równań liniowych minimalizującego błąd średniokwadratowy:
$$
\begin{align}
        \begin{pmatrix}
        x_1 & 1 \\
        x_2 & 1 \\
        \ldots & \ldots \\
        x_N & 1
        \end{pmatrix}
        \begin{pmatrix}
        a  \\
        b  \\
        \end{pmatrix}
        =        
        \begin{pmatrix}
        y_1  \\
        y_2  \\
        \ldots \\
        y_N
        \end{pmatrix}
    \end{align}
$$

Do rozwiązania wykorzystamy funkcję `torch.linalg.lstsq` znajdującej przybliżone rozwiązanie nadokreślonego układu równań liniowych minimalizujące błąd średniokwadratowy. Patrz: [link](https://pytorch.org/docs/stable/generated/torch.linalg.lstsq.html#torch-linalg-lstsq).


In [ ]:
n = len(data)
x_values, y_values = zip(*data)

x = torch.ones((n, 2))
x[:, 0] = torch.tensor(x_values, dtype=torch.float32)
print(f"{x=}")

y = torch.tensor(y_values, dtype=torch.float32)
print(f"{y=}")

alpha = torch.linalg.lstsq(x, y).solution
a = alpha[0].item()
b = alpha[1].item()
print(f"Wyznaczone parametry: {a=:.4f}, {b=:.4f}")

Wizualizacja danych pomiarowych i wyznaczonej prostej regresji liniowej.

In [ ]:
x_data = [e[0] for e in data]
y_data = [e[1] for e in data]

x_min = min(x_data)
x_max = max(x_data)

x_line = torch.linspace(x_min, x_max, 100)
y_line = a * x_line + b

plt.scatter(x_data, y_data, color='blue', label='Punkty pomiarowe')
plt.plot(x_line, y_line, color='green', label='')

plt.xlabel('Wiek')
plt.ylabel('Ciśnienie skurczowe')
plt.legend()
plt.grid()

plt.show()